In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from data_utility import *
from data_utils import *
from loss import *
from train import *
from deeplab_model.deeplab import *
from sync_batchnorm import convert_model
import datetime

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
USE_GPU = True
NUM_WORKERS = 12
BATCH_SIZE = 2 

dtype = torch.float32 
# define dtype, float is space efficient than double

if USE_GPU and torch.cuda.is_available():
    
    device = torch.device('cuda')
    
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    # magic flag that accelerate
    
    print('using GPU for training')
else:
    device = torch.device('cpu')
    print('using CPU for training')

using GPU for training


In [3]:
train_dataset = pyramid_dataset(data_type = 'nii_train', 
                transform=transforms.Compose([
                random_affine(90, 15),
                random_filp(0.5)]))
# do data augumentation on train dataset

validation_dataset = pyramid_dataset(data_type = 'nii_test', 
                transform=None)
# no data augumentation on validation dataset

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS) # drop_last
# loaders come with auto batch division and multi-thread acceleration

In [ ]:
'''
deeplab = DeepLabModified(output_stride=8, elu=True)
deeplab = nn.DataParallel(deeplab)
deeplab = convert_model(deeplab)
deeplab = deeplab.to(device=device, dtype=dtype)
#shape_test(icnet1, True)
# create the model, by default model type is float, use model.double(), model.float() to convert
# move the model to desirable device

optimizer = optim.Adam(deeplab.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
epoch = 0
'''

# create an optimizer object
# note that only the model_2 params and model_4 params will be optimized by optimizer

"\ndeeplab = DeepLabModified(output_stride=8, elu=True)\ndeeplab = nn.DataParallel(deeplab)\ndeeplab = convert_model(deeplab)\ndeeplab = deeplab.to(device=device, dtype=dtype)\n#shape_test(icnet1, True)\n# create the model, by default model type is float, use model.double(), model.float() to convert\n# move the model to desirable device\n\noptimizer = optim.Adam(deeplab.parameters(), lr=1e-2)\nscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)\nepoch = 0\n"

In [ ]:

deeplab = DeepLabModified(output_stride=8, elu=True)
deeplab = nn.DataParallel(deeplab)
deeplab = convert_model(deeplab)

#checkpoint = torch.load('../deeplab_save/2019-07-29 04:00:14.630172.pth') # second best
#checkpoint = torch.load('../deeplab_save/2019-07-28 23:47:36.279119.pth') # second best
#checkpoint = torch.load('../deeplab_save/2019-07-29 00:15:49.271222.pth') # best
#checkpoint = torch.load('../deeplab_save/2019-07-29 00:44:11.825872.pth')
checkpoint = torch.load('../deeplab_output_8_modified_elu_save/2019-08-26 08:55:53.958624 epoch: 460.pth') # latest one

deeplab.load_state_dict(checkpoint['state_dict_1'])
deeplab = deeplab.to(device=device, dtype=dtype)

optimizer = optim.Adam(deeplab.parameters(), lr=1e-2)
optimizer.load_state_dict(checkpoint['optimizer'])

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)
scheduler.load_state_dict(checkpoint['scheduler'])

epoch = checkpoint['epoch']
print(epoch)
for param_group in optimizer.param_groups:
    print(param_group['lr'])


460
0.01


In [ ]:
epochs = 5000

min_val = 1

record = open('train_deeplab_output_8_modified_elu.txt','a+')

logger = {'train':[], 'validation_1': []}

for e in tqdm(range(epoch + 1, epochs)):
# iter over epoches

    epoch_loss = 0
        
    for t, batch in enumerate(train_loader):
    # iter over the train mini batches
    
        deeplab.train()
        # Set the model flag to train
        # 1. enable dropout
        # 2. batchnorm behave differently in train and test
        
        image_1 = batch['image1_data'].to(device=device, dtype=dtype)
        label_1 = batch['image1_label'].to(device=device, dtype=dtype)
        # move data to device, convert dtype to desirable dtype
        
        out_1 = deeplab(image_1)
        # do the inference
        
        del image_1
        torch.cuda.empty_cache()

        loss_1 = dice_loss_3(out_1, label_1)
        # calculate loss
        
        epoch_loss += loss_1.item()
        # record minibatch loss to epoch loss
        
        optimizer.zero_grad()
        # set the model parameter gradient to zero
        
        loss_1.backward()
        # calculate the gradient wrt loss
        optimizer.step()
        #scheduler.step(loss_1)
        # take a gradient descent step
        
    outstr = 'Epoch {0} finished ! Training Loss: {1:.4f}'.format(e, epoch_loss/(t+1)) + '\n'
    
    logger['train'].append(epoch_loss/(t+1))
    
    print(outstr)
    record.write(outstr)
    record.flush()
    
    if (e <= 150 and e%5 == 0) or (e > 150 and e <= 550 and e%3 == 0) or (e > 550 and e%1 == 0):
        # do validation every 5 epoches

            deeplab.eval()
            # set model flag to eval
            # 1. disable dropout
            # 2. batchnorm behave differs

            with torch.no_grad():
            # stop taking gradient

                #valloss_4 = 0
                #valloss_2 = 0
                valloss_1 = 0

                for v, vbatch in enumerate(validation_loader):
                # iter over validation mini batches

                    image_1_val = vbatch['image1_data'].to(device=device, dtype=dtype)
                    if get_dimensions(image_1_val) == 4:
                        image_1_val.unsqueeze_(0)
                    label_1_val = vbatch['image1_label'].to(device=device, dtype=dtype)
                    if get_dimensions(label_1_val) == 4:
                        label_1_val.unsqueeze_(0)
                    # move data to device, convert dtype to desirable dtype
                    # add one dimension to labels if they are 4D tensors

                    out_1_val = deeplab(image_1_val)
                    del image_1_val
                    torch.cuda.empty_cache()
                    # do the inference

                    loss_1 = dice_loss_3(out_1_val, label_1_val)
                    # calculate loss

                    valloss_1 += loss_1.item()
                    # record mini batch loss

                avg_val_loss = (valloss_1 / (v+1))
                outstr = '------- 1st valloss={0:.4f}'\
                    .format(avg_val_loss) + '\n'

                logger['validation_1'].append(avg_val_loss)
                #scheduler.step(avg_val_loss)

                print(outstr)
                record.write(outstr)
                record.flush()

                if avg_val_loss < min_val:
                    print(avg_val_loss, "less than", min_val)
                    min_val = avg_val_loss
                    save_1('deeplab_output_8_modified_elu_save', deeplab, optimizer, logger, e, scheduler)
                elif e%10 == 0:
                    save_1('deeplab_output_8_modified_elu_save', deeplab, optimizer, logger, e, scheduler)
                    
                torch.cuda.empty_cache()

record.close()

  0%|          | 1/4539 [16:41<1263:00:17, 1001.94s/it]

Epoch 461 finished ! Training Loss: 0.1350

Epoch 462 finished ! Training Loss: 0.1292

------- 1st valloss=0.1182

0.11819245018388914 less than 1


  0%|          | 2/4539 [31:53<1228:27:56, 974.76s/it] 

Checkpoint 462 saved !


  0%|          | 3/4539 [46:20<1187:39:39, 942.59s/it]

...

------- 1st valloss=0.0537

Epoch 984 finished ! Training Loss: 0.0539



 12%|█▏        | 524/4539 [133:29:08<1031:20:09, 924.73s/it]

------- 1st valloss=0.1172

Epoch 985 finished ! Training Loss: 0.0522



 12%|█▏        | 525/4539 [133:44:27<1029:06:21, 922.96s/it]

------- 1st valloss=0.1286



In [ ]:
deeplab.eval()

with torch.no_grad():
    
    bgloss = 0
    bdloss = 0
    bvloss = 0
    
    for v, vbatch in tqdm(enumerate(validation_loader)):
            # move data to device, convert dtype to desirable dtype

        image_1 = vbatch['image1_data'].to(device=device, dtype=dtype)
        label_1 = vbatch['image1_label'].to(device=device, dtype=dtype)

        output = deeplab(image_1)
        # do the inference
        output_numpy = output.cpu().numpy()
        
        
        #out_1 = torch.round(output)
        out_1 = torch.from_numpy((output_numpy == output_numpy.max(axis=1)[:, None]).astype(int)).to(device=device, dtype=dtype)
        loss_1 = dice_loss_3(out_1, label_1)

        bg, bd, bv = dice_loss_3_debug(out_1, label_1)
        # calculate loss
        print(bg.item(), bd.item(), bv.item(), loss_1.item())
        bgloss += bg.item()
        bdloss += bd.item()
        bvloss += bv.item()

    outstr = '------- background loss = {0:.4f}, body loss = {1:.4f}, bv loss = {2:.4f}'\
        .format(bgloss/(v+1), bdloss/(v+1), bvloss/(v+1)) + '\n'
    print(outstr)